## Model development for Legacy model
- Train word embeddings from scratch

In [1]:
!pip install nltk
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.8 MB/s eta 0:00:00a 0:00:010m

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from build_model import *

# features = feature_extractions("Flickr8k_Dataset")

# #save features for future use.
# with open("features.pkl", "wb") as f:
#    dump(features, f)

ModuleNotFoundError: No module named 'build_model'

In [ ]:
with open("Flickr8k_text/Flickr8k.token.txt", "r") as f:
    raw_caption = f.read()

caption_dict = caption_dictionary(raw_caption)
caption_dict = caption_cleaning(caption_dict)

#Save the caption_dict for future use
with open("captions.pkl", "wb") as f:
    dump(caption_dict, f)

In [ ]:
train = dataset("Flickr8k_text/Flickr_8k.trainImages.txt")
dev = dataset("Flickr8k_text/Flickr_8k.devImages.txt")
test = dataset("Flickr8k_text/Flickr_8k.testImages.txt")

train_caption_dict = load_captions(train, 1)
train_features = load_features(train)
dev_caption_dict = load_captions(dev, 1)
dev_features = load_features(dev)
test_caption_dict = load_captions(test, 0)
test_features = load_features(test)

#Initialize the tokenizer
vocab_size = 3001
tokenizer = create_tokenizer(train_caption_dict, num_vocab=vocab_size)

#Save the tokenizer for caption_generation
with open("../tokenizer.json", "w") as f:
    json.dump(tokenizer.to_json(), f)

print("vocab size: {}".format(vocab_size))
max_length = max([len(c.split()) for c in caption_to_list(train_caption_dict)]) - 1 #Maximum length of input sequence
print("max length: {}".format(max_length))

In [ ]:
path_to_glove_file = "./glove.6B/glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file, 'r', encoding='utf-8') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

#Initialize embedding matrix
embed_dim = 100
embedding_matrix = np.zeros((vocab_size, embed_dim))
coverage = set()
for i in range(1, vocab_size):
    w = tokenizer.index_word[i]
    if w in embeddings_index.keys():
        embedding_matrix[i] = embeddings_index.get(w) #use the pre-trained embeddings, set 0 otherwise
        coverage.add(w)

oov = check_coverage(tokenizer, coverage, vocab_size=vocab_size)
print(oov)

In [ ]:
num_epoches = 10
num_photos = 32 #photos per batch

steps_per_epoch = int(len(train_caption_dict) / num_photos)
val_steps = int(len(dev_caption_dict) / num_photos)

model = define_model(max_length, vocab_size, dp_rate=0.1, embed_size=embed_dim, embedding_matrix=None)

train_generator = generate_dataset(train_caption_dict, train_features, tokenizer, max_length, vocab_size, num_photos)
dev_generator = generate_dataset(dev_caption_dict, dev_features, tokenizer, max_length, vocab_size, num_photos)

for i in range(num_epoches):
    for step in range(5*steps_per_epoch):
        X, Y = next(train_generator)
        model.train_on_batch(X, Y)
    
    model.save("../model/model_v{i}.h5".format(i=i))
    print("finish saving model: model_v{i}".format(i=i))

In [ ]:
%%time
evaluate_model(model, tokenizer, test_caption_dict, test_features, max_length, vocab_size)